# Lean 4 Compilation Test

Test if Lean code compiles correctly using our lean server.

In [1]:
import sys
sys.path.insert(0, '../src')

from utils.lean_utils import create_lean_server, verify_with_lean

In [2]:
# Create Lean server
lean_server = create_lean_server()
print('Lean server created')

Lean server created


In [3]:
# Test code
lean_code = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → Nice x
theorem cat_nice : Nice Cat := R1 Cat T1
'''

print('Testing code:')
print(lean_code)

Testing code:

axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → Nice x
theorem cat_nice : Nice Cat := R1 Cat T1



In [4]:
# Verify
result = verify_with_lean(lean_code, lean_server, verbose=True)

print('\n=== Result ===')
print(f"Success: {result['success']}")
print(f"Errors: {result['errors']}")
print(f"Warnings: {result['warnings']}")


Verifying Lean code:

axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → Nice x
theorem cat_nice : Nice Cat := R1 Cat T1


Verification succeeded

=== Result ===
Success: True
Errors: []
Warnings: []


## Example 1: True

**Problem:**
- Context: The cat is blue. If someone is blue then they are nice.
- Question: The cat is nice.
- Answer: True (because we proved Nice Cat)

## Example 2: Uncertain

**Problem:**
- Context: The cat is blue. If someone is nice then they are red.
- Question: The cat is red.
- Answer: Uncertain (cannot prove Red Cat or ¬ Red Cat from given axioms)

In [ ]:
# Example 2: Uncertain case
lean_code_uncertain = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom Red : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Nice x → Red x
-- Cannot prove Red Cat or ¬ Red Cat from given axioms
'''

print('Testing Uncertain case:')
print(lean_code_uncertain)

result = verify_with_lean(lean_code_uncertain, lean_server, verbose=True)
print('\n=== Result ===')
print(f"Success: {result['success']}")
print(f"Errors: {result['errors']}")
print(f"Warnings: {result['warnings']}")

## Example 3: False

**Problem:**
- Context: The cat is blue. If someone is blue then they are not red.
- Question: The cat is red.
- Answer: False (because we proved ¬Red Cat)

In [ ]:
# Example 3: False case
lean_code_false = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Red : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → ¬Red x
theorem cat_not_red : ¬Red Cat := R1 Cat T1
'''

print('Testing False case:')
print(lean_code_false)

result = verify_with_lean(lean_code_false, lean_server, verbose=True)
print('\n=== Result ===')
print(f"Success: {result['success']}")
print(f"Errors: {result['errors']}")
print(f"Warnings: {result['warnings']}")

## Bidirectional Prompts Test

Testing examples from bidirectional prompts to ensure they compile correctly.

### Bidirectional True - Example 1 (Success)

From `prompts/bidirectional/true_system.txt`

In [ ]:
# Bidirectional True - Example 1 (Success)
# Context: The cat is blue. If someone is blue then they are nice.
# Question: The cat is nice.

bidir_true_ex1 = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → Nice x
theorem cat_nice : Nice Cat := R1 Cat T1
'''

print('Bidirectional True - Example 1:')
result = verify_with_lean(bidir_true_ex1, lean_server, verbose=True)
print(f"\nSuccess: {result['success']}")

In [ ]:
# Bidirectional True - Example 2 (Failure)
# Context: The cat is blue. If someone is nice then they are red.
# Question: The cat is red.
# Cannot prove Red Cat from given axioms

bidir_true_ex2 = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom Red : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Nice x → Red x
-- Cannot prove Red Cat from given axioms
'''

print('Bidirectional True - Example 2 (Failure case):')
result = verify_with_lean(bidir_true_ex2, lean_server, verbose=True)
print(f"\nSuccess: {result['success']}")

In [ ]:
# Bidirectional False - Example 1 (Success)
# Context: The cat is blue. If someone is blue then they are not red.
# Question: The cat is red.
# Proves ¬Red Cat

bidir_false_ex1 = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Red : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → ¬Red x
theorem cat_not_red : ¬Red Cat := R1 Cat T1
'''

print('Bidirectional False - Example 1:')
result = verify_with_lean(bidir_false_ex1, lean_server, verbose=True)
print(f"\nSuccess: {result['success']}")

In [ ]:
# Bidirectional False - Example 2 (Failure)
# Context: The cat is blue. If someone is nice then they are red.
# Question: The cat is red.
# Cannot prove ¬Red Cat from given axioms

bidir_false_ex2 = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom Red : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Nice x → Red x
-- Cannot prove ¬Red Cat from given axioms
'''

print('Bidirectional False - Example 2 (Failure case):')
result = verify_with_lean(bidir_false_ex2, lean_server, verbose=True)
print(f"\nSuccess: {result['success']}")

## Two-Stage Pipeline Examples

Testing examples from two-stage prompts.

In [ ]:
# Stage 1: Translation (axioms + theorem with sorry)
# Context: The cat is blue. If someone is blue then they are nice.
# Conclusion: The cat is nice.

stage1_example = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → Nice x
theorem goal : Nice Cat := sorry
'''

print('Stage 1 - Translation (with sorry):')
result = verify_with_lean(stage1_example, lean_server, verbose=True)
print(f"\nSuccess: {result['success']}")
print(f"Errors: {result['errors']}")

In [ ]:
# Stage 2: Proving (replace sorry with proof)
# Same axioms, but with actual proof

stage2_example = '''
axiom obj : Type
axiom Cat : obj
axiom Blue : obj → Prop
axiom Nice : obj → Prop
axiom T1 : Blue Cat
axiom R1 : ∀ x : obj, Blue x → Nice x
theorem goal : Nice Cat := R1 Cat T1
'''

print('Stage 2 - Proving (with proof):')
result = verify_with_lean(stage2_example, lean_server, verbose=True)
print(f"\nSuccess: {result['success']}")
print(f"Errors: {result['errors']}")